In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, get_train, get_test
import session_new as session

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
sub_type = 'TR'
with open('train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,_,_ = pickle.load(f)

n_subs = np.max(params[:,0])

In [3]:
train_sess = {'sub_type':sub_type,'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'all','scaler_load':False,'feat_type':'feat','epochs':30}
test_sess = {'test_grp':4, 'test':'partposrealmixeven24'}
sess = session.Sess(**train_sess)
sess.update(**test_sess)
n_subs = 2#np.max(params[:,0])
data_load = True
test = False
train = True

for sub in range(1,n_subs+1):
    print('Running ' + str(sess.sub_type) + str(sub))
    sess.sub = sub
    tf.keras.backend.clear_session()

    train_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.train_grp)
    test_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.test_grp)

    n_dof = np.max(params[train_ind,4])
    
    # Train NNs
    mlp = MLP(n_class=n_dof)
    mlp_beta = MLPbeta(n_class=n_dof)
    cnn = CNN(n_class=n_dof)

    optimizer = tf.keras.optimizers.Adam()

    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
    
    if np.sum(train_ind) and np.sum(test_ind):

        if train:
            if data_load:
                with open('noisedata_all_emgscalelim/' + str(sess.sub_type) + str(sub) + '_grp_2_fullallmix4_5.p','rb') as f:
                    scaler, x_train_cnn, _, _, _, y_train, _, x_train_lda, y_train_lda, x_train_aug, y_train_aug = pickle.load(f)
            
                x_train_cnn = x_train_cnn.astype('float32')
                x_train_mlp = x_train_cnn.reshape(x_train_cnn.shape[0],-1)
                
                trainmlp = tf.data.Dataset.from_tensor_slices((x_train_mlp, y_train)).shuffle(x_train_mlp.shape[0],reshuffle_each_iteration=True).batch(128)
                traincnn = tf.data.Dataset.from_tensor_slices((x_train_cnn, y_train)).shuffle(x_train_cnn.shape[0],reshuffle_each_iteration=True).batch(128)
            
                sess.scaler = scaler
            else:   
                trainmlp, validmlp, traincnn, validcnn, y_train, y_valid, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, y_train_aug = prd.prep_train_data(sess,raw,params)

            # Train neural networks
            models = [mlp, mlp_beta, cnn]
            for model in models:

                if isinstance(model,CNN):
                    ds = traincnn
                else:
                    ds = trainmlp
                
                train_mod = get_train()

                for epoch in range(sess.epochs):
                    # Reset the metrics at the start of the next epoch
                    train_loss.reset_states()
                    train_accuracy.reset_states()

                    for x, y in ds:
                        train_mod(x, y, model, optimizer, train_loss, train_accuracy)

                    if epoch == 0 or epoch == sess.epochs-1:
                        print(
                            f'Epoch {epoch + 1}, '
                            f'Loss: {train_loss.result():.2f}, '
                            f'Accuracy: {train_accuracy.result() * 100:.2f} '
                        )
                
                del train_mod
        
            # Train aligned LDA
            y_train_aligned = np.argmax(y_train, axis=1)[...,np.newaxis]

            mlp_enc = mlp.get_layer(name='enc')
            w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aligned)

            mlpbeta_enc = mlp_beta.get_layer(name='enc')
            w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aligned)
            
            cnn_enc = cnn.get_layer(name='enc')
            temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
            temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
            w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aligned)

            # Train LDA
            w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
            w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

            mlp_w = mlp.get_weights()
            mlpb_w = mlp_beta.get_weights()
            cnn_w = cnn.get_weights()

            with open('models/' + str(sess.sub_type) + str(sub) + '_' + str(sess.feat_type) + '.p','wb') as f:
                pickle.dump([mlp_w, mlpb_w, cnn_w, w_mlp, c_mlp, w_mlpbeta, c_mlpbeta, w_cnn, c_cnn, w, c, w_aug, c_aug, sess.emg_scale, sess.scaler],f)
        
        else:
            with open('models/' + str(sess.sub_type) + str(sub) + '_' + str(sess.feat_type) + '.p','rb') as f:
                mlp_w, mlpb_w, cnn_w, w_mlp, c_mlp, w_mlpbeta, c_mlpbeta, w_cnn, c_cnn, w, c, w_aug, c_aug, sess.emg_scale, sess.scaler = pickle.load(f)

            mlp_enc = mlp.get_layer(name='enc')
            mlpbeta_enc = mlp_beta.get_layer(name='enc')
            cnn_enc = cnn.get_layer(name='enc')

        if test:
            # Load test data
            if data_load:
                _, x_test, _, _, _, _ = prd.train_data_split(raw,params,sub,sess.sub_type,dt=sess.cv_type,train_grp=sess.test_grp)
                clean_size = int(np.size(x_test,axis=0))

                with open('testdata_all_emgscalelim_noisescalelim/TR' + str(sub) + '_grp_2_' + str(sess.test) + '_1.p','rb') as f:
                    x_test_cnn, _, x_test_lda, y_test = pickle.load(f) 
                x_temp = np.transpose(x_test_lda.reshape((x_test_lda.shape[0],4,-1)),(0,2,1))[...,np.newaxis]
                x_test_cnn = sess.scaler.transform(x_temp.reshape(x_temp.shape[0]*x_temp.shape[1],-1)).reshape(x_temp.shape)
                x_test_cnn = x_test_cnn.astype('float32')

                # Reshape for nonconvolutional SAE
                x_test_mlp = x_test_cnn.reshape(x_test_cnn.shape[0],-1)
            else:
                with open('real_noise/all_real_noise.p', 'rb') as f:
                    real_noise_temp, _ = pickle.load(f)

                x_test_cnn, x_test_mlp, x_test_lda, y_test, clean_size = prd.prep_test_data(sess, raw, params, real_noise_temp)

            # workaround
            mlp(x_test_mlp[:2,...])
            mlp_beta(x_test_mlp[:2,...])
            cnn(x_test_cnn[:2,...])

            mlp.set_weights(mlp_w)
            mlp_beta.set_weights(mlpb_w)
            cnn.set_weights(cnn_w)
        
            # Test
            mlp_test_aligned = mlp_enc(x_test_mlp).numpy()
            mlpbeta_test_aligned = mlpbeta_enc(x_test_mlp).numpy()
            cnn_test_aligned = cnn_enc(x_test_cnn).numpy()
            y_test_aligned = np.argmax(y_test, axis=1)[...,np.newaxis]

            # LDA
            clean_lda, noisy_lda = eval_lda(w, c, x_test_lda, y_test_aligned, clean_size)
            # AUG-LDA
            clean_aug, noisy_aug = eval_lda(w_aug, c_aug, x_test_lda, y_test_aligned, clean_size)
            # MLP
            clean_mlp, noisy_mlp = eval_nn(x_test_mlp, y_test,mlp,clean_size)
            clean_mlplda, noisy_mlplda = eval_lda(w_mlp, c_mlp, mlp_test_aligned, y_test_aligned, clean_size)
            # MLP Beta
            clean_mlpb, noisy_mlpb = eval_nn(x_test_mlp, y_test,mlp_beta,clean_size)
            clean_mlpblda, noisy_mlpblda = eval_lda(w_mlpbeta, c_mlpbeta, mlpbeta_test_aligned, y_test_aligned, clean_size)
            # CNN
            clean_cnn, noisy_cnn = eval_nn(x_test_cnn, y_test,cnn,clean_size)
            clean_cnnlda, noisy_cnnlda = eval_lda(w_cnn, c_cnn, cnn_test_aligned, y_test_aligned, clean_size)

            print( 
                f'LDA ---- '
                f'Clean: {clean_lda * 100:.2f}, '
                f'Noisy: {noisy_lda * 100:.2f}'
                f'\nAUG ---- '
                f'Clean: {clean_aug * 100:.2f}, '
                f'Noisy: {noisy_aug * 100:.2f}'
                f'\nMLP ---- '
                f'Clean: {clean_mlp * 100:.2f}, '
                f'Noisy: {noisy_mlp * 100:.2f}, '
                f'LDA Clean: {clean_mlplda * 100:.2f}, '
                f'LDA Noisy: {noisy_mlplda * 100:.2f}'
                f'\nMLPB ---- '
                f'Clean: {clean_mlpb * 100:.2f}, '
                f'Noisy: {noisy_mlpb * 100:.2f}, '
                f'LDA Clean: {clean_mlpblda * 100:.2f}, '
                f'LDA Noisy: {noisy_mlpblda * 100:.2f}'
                f'\nCNN ---- '
                f'Clean: {clean_cnn * 100:.2f}, '
                f'Noisy: {noisy_cnn * 100:.2f}, '
                f'LDA Clean: {clean_cnnlda * 100:.2f}, '
                f'LDA Noisy: {noisy_cnnlda * 100:.2f}'
            )

            clean_all = np.stack((clean_lda,clean_aug,clean_mlp,clean_mlplda,clean_mlpb,clean_mlpblda,clean_cnn,clean_cnnlda))
            noisy_all = np.stack((noisy_lda,noisy_aug,noisy_mlp,noisy_mlplda,noisy_mlpb,noisy_mlpblda,noisy_cnn,noisy_cnnlda))
    else:
        print('no training or testing data')

Epoch 1, Loss: 1.86, Accuracy: 26.01 
Epoch 30, Loss: 0.88, Accuracy: 66.57 
Epoch 1, Loss: 1.22, Accuracy: 54.43 
Epoch 30, Loss: 0.38, Accuracy: 85.60 
Epoch 1, Loss: 1.18, Accuracy: 57.41 
Epoch 30, Loss: 0.32, Accuracy: 87.91 
Epoch 1, Loss: 1.77, Accuracy: 28.43 
Epoch 30, Loss: 0.66, Accuracy: 72.98 
Epoch 1, Loss: 1.11, Accuracy: 57.39 
Epoch 30, Loss: 0.26, Accuracy: 89.87 
Epoch 1, Loss: 1.00, Accuracy: 65.32 


KeyboardInterrupt: 